In [ ]:
num_epochs = 50
batch_size = 10
best_loss = float('inf')
patience = 0
lr_decay = 0.9

train_losses = []
val_losses = []
optimizer = optim.Adam(model.parameters(), lr=0.001)



for epoch in range(num_epochs):
    running_loss = 0.0
    model.train()

    for i in range(0, len(train_img), batch_size):
        batch_img = train_img[i:i+batch_size].to(device)
        batch_label = train_label[i:i+batch_size].to(device)

        optimizer.zero_grad()
        output = model(batch_img)
        mse = nn.MSELoss()(output, batch_label)
        perceptual = perceptual_loss(output, batch_label)
        total_loss = mse + 0.01 * perceptual

        total_loss.backward()
        optimizer.step()

        running_loss += total_loss.item()

    avg_train_loss = running_loss / (len(train_img) // batch_size)

    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for i in range(0, len(val_img), batch_size):
            batch_img = val_img[i:i+batch_size].to(device)
            batch_label = val_label[i:i+batch_size].to(device)

            output = model(batch_img)
            mse = nn.MSELoss()(output, batch_label)
            perceptual = perceptual_loss(output, batch_label)
            total_loss = mse + 0.01 * perceptual

            val_loss += total_loss.item()

    avg_val_loss = val_loss / (len(val_img) // batch_size)

    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        patience = 0
        torch.save(model.state_dict(), '/content/drive/MyDrive/DEEP/models/best_model.pt')
    else:
        patience += 1
        if patience >= 2:
            for param_group in optimizer.param_groups:
                param_group['lr'] *= lr_decay
            patience = 0

    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    print(f"Epoch {epoch+1}/{num_epochs}, Learning Rate: {optimizer.param_groups[0]['lr']:.6f}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")

model.load_state_dict(torch.load('/content/drive/MyDrive/DEEP/models/best_model.pt'))
torch.save(model.state_dict(), 'srresnet_model_vgg.pt')